# Docker + Postgres Notes
### This notebook contains helpful cmds and snippets for:
* Creating docker containers with images such as pgadmin and postgres13
* Configuring a network for communication betweeen containers
* Creating local volume mounts for data persistence between container lifecycles

In order to see what is currently running on a `<port>`:

```
sudo lsof -i :<port>
```
In order to kill a process that is currently running:
```
kill <pid>
```

*Note: Running the first cmd exposes the PID*

To create docker network:


```
docker network create --name
```

In order to create docker image with locally instantiated docker volume:
```
docker run -it -e POSTGRES_USER="root" -e POSTGRES_PASSWORD="root" -e POSTGRES_DB="ny_taxi" -v dtc_postgres_volume_local:/var/lib/postgresql/data -p 5432:5432 postgres:13
```

In order to interact with docker image created above:

```
pgcli -h localhost -p 5432 -u root -d ny_taxi
```

In order to convert ipynb to .py script

```
jupyter nbconvert --to=script {NAME_OF_NOTEBOOK}.ipynb
```

In order to create Postgres Database instance on a docker network.

```
docker run -it -e POSTGRES_USER="root" -e POSTGRES_PASSWORD="root" -e POSTGRES_DB="ny_taxi" -v dtc_postgres_volume_local:/var/lib/postgresql/data -p 5432:5432 --network=pg-network --name {DB_NAME} postgres:13
```

In order to create Pgadmin instance on a docker network.


```
docker run -it -e PGADMIN_DEFAULT_EMAIL="admin@admin.com" -e PGADMIN_DEFAULT_PASSWORD="root" -p 8080:80 --network=pg-network --name pg-admin dpage/pgadmin4
```

In order to run .py file with associated arguments, captured with argparse

```
python data-migration-postgres.py --user=root --password=root --host=localhost --port=5432 --db=ny_taxi --table_name=yellow_taxi_trips --url=${URL}
```

In order to build data-migration:v001 container with Dockerfile in root project directory:

```
docker build -t data-migration:v001 .
```

Here is a Dockerfile with specifications for pip installation, file generation, and execution flow for the container you created with the above build command
```
FROM python:3.9

RUN pip install pandas sqlalchemy psycopg2 fastparquet

WORKDIR /app
COPY data-migration-postgres.py data-migration-postgres.py

ENTRYPOINT [ "python", "data-migration-postgres.py" ]
```

Define bash variable URL and then run previously created docker container and the shared docker network:

```
URL="https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2021-01.parquet"
```
```
docker run -it --network=pg-network data-migration:v001 --user=root --password=root --host={DB_NAME} --port=5432 --db=ny_taxi --table_name=yellow_taxi_trips --url=${URL}
```

Example docker-compose that creates pgadmin container and postgres container on a shared network, with specified local volumes responsible for persisting data across runs. (Make sure to create volumes prior to running)

```
services:
  pgdatabase:
    image: postgres:13
    environment:
      - POSTGRES_USER=root
      - POSTGRES_PASSWORD=root
      - POSTGRES_DB=ny_taxi
    volumes: 
      - "./source/volumes/pgdatabase:/var/lib/postgresql/data:rw"
    ports:
      - "5432:5432"
  pgadmin:
    image: dpage/pgadmin4
    environment:
      - PGADMIN_DEFAULT_EMAIL=admin@admin.com
      - PGADMIN_DEFAULT_PASSWORD=root
    volumes:
      - "./source/volumes/pgadmin:/var/lib/pgadmin:rw"
    ports:
      - "8080:80"
networks:
  default:
    name: pg-network
```

To change the permissions of a directory (in order to use it as a local storage for data persistence between container runs)

```
sudo chown 5050:5050 data_pgadmin
```

Docker CMDS:

```
docker ps | Lists all currently active docker containers
docker volume ls | Lists all currently active .docker/volumes
docker-compose up -d | Runs docker-compose to create containeres on shared network in detached mode to free up the CLI
docker-compose down | Shuts down docker-compose containers on network if currently live
docker volume rm {NAME_OF_VOLUME}
docker logs {NAME OF RUNNING DOCKER CONTAINER}
```